In [2]:
import os
import pandas as pd
from docxtpl import DocxTemplate
from customtkinter import *
from customtkinter import filedialog
from CTkMessagebox import CTkMessagebox
from CTkMenuBar import *
from pandastable import Table

# Globals
df = None
tpl = None
context = {}
a = None

# Load CSV
def open_file():
    global df
    file_path = filedialog.askopenfilename(
        defaultextension='.csv',
        filetypes=[("CSV File", "*.csv"), ("All Files", "*.*")]
    )
    if file_path:
        try:
            df = pd.read_csv(file_path)
            df.columns = df.columns.str.strip()
            df.set_index('ad_no', inplace=True)
            entry.configure(state='normal')
            btn2.configure(state='normal')
            entry.focus()
        except Exception:
            CTkMessagebox(title="Error", message="Couldn't read file", icon="prohibition.png", option_1="Ok")

# Validate and enable buttons
def btn_state():
    val = entry.get().strip()
    if val.isdigit() and int(val) in df.index:
        butn1.configure(state="normal")
        butn2.configure(state="normal")
    else:
        butn1.configure(state="disabled")
        butn2.configure(state="disabled")

# Prepare context
def gen():
    global tpl, context, a
    try:
        a = int(entry.get())
        tpl = DocxTemplate("templatee.docx")
        row = df.loc[a]
        context = {
            "name": row['Name'],
            "course": row['Course'],
            "modules": row['Modules'],
            "start_date": row['Starting Date'],
            "Comp_date": row['Completion Date'],
            "course_dur": row['Course Duration']
        }
        return True
    except Exception:
        CTkMessagebox(title="Error", message="Invalid admission number", icon="prohibition.png", option_1="Ok")
        return False

# Save certificate
def save_doc():
    global tpl, context, a
    file_path = filedialog.asksaveasfilename(
        initialfile=f"{a} - {df.loc[a, 'Name']}.docx",
        defaultextension=".docx",
        filetypes=[("Word Documents", "*.docx"), ("All Files", "*.*")]
    )
    if file_path:
        try:
            tpl.render(context)
            tpl.save(file_path)
            CTkMessagebox(title="File Saved", message=f"{a} - {context['name']} saved", icon="check (1).png", option_1="Ok")
        except Exception as e:
            CTkMessagebox(title="Error", message=f"Couldn't save file: {e}", icon="prohibition.png", option_1="Ok")

# Combined generate + save
def filegen():
    if gen():
        save_doc()

# Show profile info
def profile():
    if not gen():
        return
    data = pd.DataFrame(df.loc[a]).reset_index()
    top = CTkToplevel(root)
    top.title(f"{a} - {context['name']}")
    frame = CTkFrame(top)
    frame.pack(fill='both', expand=True)
    Table(frame, dataframe=data).show()

# View full CSV
def view_csv():
    top = CTkToplevel(root)
    top.title("CSV Viewer")
    top.geometry("800x600")
    CTkMenuBar(top).add_cascade(text="Print All", command=print_all)
    frame = CTkFrame(top)
    frame.pack(fill='both', expand=True)
    data1=df.reset_index()
    Table(frame, dataframe=data1).show()

# Bulk certificate generation
def print_all():
    folder_path = filedialog.askdirectory(title="Select folder to save certificates")
    if not folder_path:
        CTkMessagebox(title="Cancelled", message="No folder selected.", icon="prohibition.png")
        return

    for ad_no in df.index:
        try:
            tpl = DocxTemplate(r"D:\vscode\cert\templatee.docx")
            row = df.loc[ad_no]
            context = {
                "name": row['Name'],
                "course": row['Course'],
                "modules": row['Modules'],
                "start_date": row['Starting Date'],
                "Comp_date": row['Completion Date'],
                "course_dur": row['Course Duration']
            }
            tpl.render(context)
            filename = f"{ad_no} - {row['Name']}.docx"
            tpl.save(os.path.join(folder_path, filename))
        except Exception as e:
            print(f"Error saving {ad_no}: {e}")

    CTkMessagebox(title="File Saved", message="All certificates saved successfully.", icon="check (1).png")

# GUI setup
root = CTk()
root.title("Certificate Generator")
root.geometry('350x150')

CTkLabel(root, text="Admission number:").grid(row=2, column=0, padx=10, pady=10)

entry = CTkEntry(root, state='disabled')
entry.grid(row=2, column=1, padx=10, pady=10)
entry.bind("<KeyRelease>", lambda event: btn_state())

butn1 = CTkButton(root, text="print", state='disabled', command=filegen)
butn1.grid(row=3, column=0, padx=10, pady=10)

butn2 = CTkButton(root, text="preview", state='disabled', command=profile)
butn2.grid(row=3, column=1, padx=10, pady=10)

btn1 = CTkButton(root, text="+.csv", width=50, height=20, command=open_file)
btn1.grid(row=4, column=0, sticky="sw", padx=10, pady=10)

btn2 = CTkButton(root, text="view CSV", state='disabled', width=50, height=20, command=view_csv)
btn2.grid(row=4, column=1, sticky="sw", padx=10, pady=10)

root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\moham\anaconda3\Lib\tkinter\__init__.py", line 2068, in __call__
    return self.func(*args)
           ~~~~~~~~~^^^^^^^
  File "c:\Users\moham\anaconda3\Lib\site-packages\customtkinter\windows\widgets\ctk_button.py", line 554, in _clicked
    self._command()
    ~~~~~~~~~~~~~^^
  File "C:\Users\moham\AppData\Local\Temp\ipykernel_25336\4016944700.py", line 132, in print_all
    CTkMessagebox(title="File Saved", message="All certificates saved successfully.", icon="check (1).png")
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\moham\anaconda3\Lib\site-packages\CTkMessagebox\ctkmessagebox.py", line 236, in __init__
    self.info = customtkinter.CTkButton(self.frame_top,  width=1, height=self.height/2, corner_radius=0, text=self.message, font=self.font,
                ~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^